In [32]:
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [34]:
train=dataset['train']
validation=dataset['validation']

In [35]:
train

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [37]:
from transformers import GPT2Tokenizer, GPT2ForQuestionAnswering, Trainer, TrainingArguments
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
train = train.map(lambda x: tokenizer(x["context"], x["question"], truncation=True), batched=True)
validation = validation.map(lambda x: tokenizer(x["context"], x["question"], truncation=True), batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [38]:
model = GPT2ForQuestionAnswering.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
output_dir = "./fine_tuned_model"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir="./logs",
)

def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [47]:
import transformers
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

# Load the SQuAD dataset (consider using a smaller subset for initial experimentation)
squad_dataset = load_dataset("squad", split="train[:1000]")  # Adjust split and size as needed

# Load pre-trained GPT2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Preprocess data for GPT2
def preprocess_function(examples):
    # Concatenate context and question with a separator token
    inputs = tokenizer(examples["context"].format("[SEP] {}", examples["question"]), return_tensors="pt")

    # Identify answer spans (if available)
    if "answer_start" in examples:
        start_positions = examples["answer_start"]
        end_positions = examples["answer_end"]
    else:
        start_positions = None
        end_positions = None

    # Create labels with a mask for padding
    label_mask = torch.ones_like(inputs["input_ids"])  # Mask for padding
    label_mask[inputs["input_ids"] == tokenizer.pad_token_id] = 0  # Set padding to zero

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "start_positions": start_positions,
        "end_positions": end_positions,
        "label_mask": label_mask,
    }

# Apply preprocessing to the dataset
squad_dataset = squad_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./outputs",  # Adjust output directory
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust number of epochs
    per_device_train_batch_size=8,  # Adjust batch size based on GPU memory
    save_steps=500,
    save_total_limit=2,  # Save only the best 2 models
)

# Create a Trainer instance with custom compute_metrics function for evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions. logits >= 0.5, predictions.predictions, np.zeros_like(predictions.predictions))

    from squad_evaluate import evaluate
    f1 = evaluate(predictions, labels)
    return {"f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=squad_dataset,
    compute_metrics=compute_metrics,
)

# Train the fine-tuned model
trainer.train()

# Save the fine-tuned model (optional)
trainer.save_model("./fine-tuned_gpt2_squad")  # Adjust save path

# How to use the fine-tuned model for question answering (example)
def answer_question(context, question):
    inputs = tokenizer(context + " [SEP] " + question, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.logits[:, :, 0]
        end_logits = outputs.logits[:, :, 1]

    start_position = torch.argmax(start_logits).item()
    end_position = torch.argmax(end_logits).item()
    answer = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_position:end_position + 1])

    return " ".join(answer)

# Example usage
context = "The Eiffel Tower is located in Paris, France."
question = "Where is the Eiffel Tower?"
answer = answer_question(context, question)
print(f"Answer: {answer}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

AttributeError: 'list' object has no attribute 'format'